In [1]:
"""
Current running version of the code

Jiri Horacek's regularized analytic continuation
see JCP 143, 184102 (2015)
"""


import argparse
import numpy
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import rac_utils as racu

In [2]:
#constants
au2eV = 27.2113845

In [3]:
"""
The main method where various guesses are made and data is parsed and selected
This box cannot be skipped
"""

res_energy = 2.5                     #this is a guess for 
res_width = 0.4                     #this is a guess for Ei
energy_max = 0.0                   #upper selection limit for E selection [0.0]
energy_min = -99.0                 #lower selection limit for E selection [-99]
pr = 1                       #print level 1-5; 5 being the highest level

                #this is the file name to be imported (CO2.dat, etc)
fname = 'CO2.dat'   
             #input data file; 1st column lambdas; then columns with energies
data = numpy.loadtxt(fname)
               

lambdas = data[:, 0]
energys = data[:, 1:]
for lam in range(len(lambdas)):
    energys[lam, :] = sorted(energys[lam, :])

In [ ]:
"""
Skip this box if the inputs do not need to be scaled
"""

energys *= au2eV           #scale input with Hartree to eV conversion factor

In [ ]:
"""
Skip this box if the binding energy convention should remain negative
"""
energys *= -1     #binding energy convention (bound states have positive EBEs)
if energy_min > 0 and energy_max >0:
    energy_max = -energy_min
    energy_min = -energy_max
elif energy_max > 0:
    energy_min = -energy_max
    energy_max = 0.0
elif energy_min > 0:
    energy_max = -energy_min
    energy_min = -99.0

In [4]:
"""
Skip this box if the data's step-potential doesn't need to be changed
"""
                #input data from step-potential: E[eV] := E[eV] - lambda[a.u.]
n_ene = numpy.shape(energys)[1]

if pr > 3:
    for i in range(n_ene):
        plt.plot(lambdas, energys[:,i], marker='d', color='orange')
    plt.title('raw data')
    plt.show()
    
for i in range(n_ene):
    energys[:, i] -= lambdas * au2eV
    
if pr > 3:
    for i in range(n_ene):
        plt.plot(lambdas, energys[:,i], marker='d', color='orange')
    plt.title('after the shift')
    plt.show()

In [5]:
"""
This box cannot be skipped 
It defines the rac function
"""

def rac(ls, Es, E_range, guess, pr):
    """ 
    main driver of R-ACCC procedure 
    """
    global ks, lbs, sigmas 
    lbs, Ebs = racu.select_energies(ls, Es, E_range, pr)
    if len(lbs) == 0:
        print ("no input data selected")
        return
    sigmas=numpy.ones(len(Es))
    """  
    Jiri's 2015 notation: E = -kappa^2     
    """
    ks = numpy.sqrt(-Ebs)
    """
    make a guess for the parameters in eq. (2)
    to do so eq.(5) must be inverted
    """
    Er=guess[0]
    G=2*guess[1]
    ag=0.5*numpy.sqrt(2.0)*(-2*Er + numpy.sqrt(4*Er**2 + G**2))**0.25
    bg=0.5*G/numpy.sqrt(-2*Er + numpy.sqrt(4*Er**2 + G**2))
    lg=lbs[0]/racu.pade_21(ks[0], [1, ag, bg], [])
    nplt=200
    kps=numpy.linspace(0,ks[-1],nplt)
    lps=numpy.zeros(nplt)
    ps = [lg, ag, bg]
    racu.graph(ks, kps, lbs, lps, ps, ["pade_21"], nplt, pr)


    """
    fit a [2,1] Pade approximant
    """ 
    if pr > 2:
        print ("------------------------------------------------")
    ps= [lg, ag, bg]
    cs= ["pade_21"]
    if pr > 2:
        print ("[2,1] Guess l, a, b = ", ps) 
    res = minimize(racu.chi, ps, args=(ks, lbs, sigmas, cs), method='BFGS', options={'gtol': 1e-6})
    ps = res.x
    if pr > 2:
        print ("[2,1] Opt   l, a, b = ", ps)
    chi2 = racu.chi(ps, ks, lbs, sigmas, cs)
    if pr > 2:
        print ("[2,1] chi^2", chi2)
    lopt=ps[0]
    aopt=ps[1]
    bopt=ps[2]
    chis = [chi2]
    racu.graph(ks, kps, lbs, lps, ps, cs, nplt, pr)
    racu.erG(aopt, bopt)

    """
    fit a [3,1] Pade approximant
    """
    if pr > 2:
        print ("------------------------------------------------")
    dg=0.0
    ps=[dg, 1, aopt, bopt]
    cs=["pade_31"]
    lg=lbs[0]/racu.pade_31(ks[0], ps, cs)
    ps=[dg, lg, aopt, bopt]
    if pr > 2:
        print ("[3,1] Guess d, l, a, b = ", ps)
    res = minimize(racu.chi, ps, args=(ks, lbs, sigmas, cs), method='BFGS', options={'gtol': 1e-6})
    ps = res.x
    if pr > 2:
        print ("[3,1] Opt   d, l, a, b = ", ps)
    chi2 = racu.chi(ps, ks, lbs, sigmas, cs)
    if pr > 2:
        print ("[3,1] chi^2", chi2)
    dopt=ps[0]
    lopt=ps[1]
    aopt=ps[2]
    bopt=ps[3]
    chis.append(chi2)
    racu.graph(ks, kps, lbs, lps, ps, cs, nplt, pr)
    racu.erG(aopt, bopt)

    """
    fit a [3,2] Pade approximant
    """
    if pr > 2:
        print ("------------------------------------------------")
    eg=0.001
    # preopt delta, epsilson, and l; keep alpha and beta fixed
    ps=[lopt, dopt, eg]
    cs=["ppade_32", aopt, bopt]
    if pr > 2:
        print ("[3,2] pre-guess l, d, e = "); print ("    ", ps)
    res = minimize(racu.chi, ps, args=(ks, lbs, sigmas, cs), method='BFGS')
    ps = res.x
    lg=ps[0]
    dg=ps[1]
    eg=ps[2]
    ps = [lg, dg, eg, aopt, bopt]
    cs = ["pade_32"]
    if pr > 2:
        print ("[3,2] pre-opt = full guess l, d, e, a, b = "); print ("    ", ps)
    res = minimize(racu.chi, ps, args=(ks, lbs, sigmas, cs), method='BFGS', options={'gtol': 1e-6})
    ps = res.x
    if pr > 2:
        print ("[3,2] full optimization    l, d, e, a, b = "); print ("    ", ps)
    chi2 = racu.chi(ps, ks, lbs, sigmas, cs)
    if pr > 2:
        print ("[3,2] chi^2", chi2)
    lopt=ps[0]
    dopt=ps[1]
    eopt=ps[2]
    aopt=ps[3]
    bopt=ps[4]
    chis.append(chi2)
    racu.graph(ks, kps, lbs, lps, ps, cs, nplt, pr)
    racu.erG(aopt, bopt)

    """
    fit a [4,2] Pade approximant
    the second resonance guess is 4*Er, 4*Gamma, 
    and the mixing factor epsilon is 1.0
    """
    if pr > 2:
        print ("------------------------------------------------")
    Ers = racu.getErs()
    Gs = racu.getGs()
    Er=Ers[-1]*20
    G=Gs[-1]*4
    wg=0.1
    gg=0.5*numpy.sqrt(2.0)*(-2*Er + numpy.sqrt(4*Er**2 + G**2))**0.25
    dg=0.5*G/numpy.sqrt(-2*Er + numpy.sqrt(4*Er**2 + G**2))
    ps = [wg, gg, dg, 1]
    cs = ["ppade_42", aopt, bopt]
    lg=lbs[0]/racu.ppade_42(ks[0], ps, cs[1:])
    # preopt gamma, delta, omega, and l; keep alpha and beta fixed
    ps=[wg, gg, dg, lg]
    if pr > 2:
        print ("[4,2] pre-guess w, g, d, l = "); print ("    ", ps[:4])
    res = minimize(racu.chi, ps, args=(ks, lbs, sigmas, cs), method='BFGS')
    ps = res.x
    wg=ps[0]
    gg=ps[1]
    dg=ps[2]
    lg=ps[3]
    ps = [wg, gg, dg, lg, aopt, bopt]
    if pr > 2:
        print ("[4,2] pre-opt = full guess w, g, d, l, a, b = "); print ("    ", ps)
    cs = ["pade_42"]
    res = minimize(racu.chi, ps, args=(ks, lbs, sigmas, cs), method='BFGS', options={'gtol': 1e-6})
    ps = res.x
    if pr > 2:
        print ("[4,2] Full optimization    w, g, d, l, a, b = "); print ("    ", ps)
    chi2 = racu.chi(ps, ks, lbs, sigmas, cs)
    if pr > 2:
        print ("[4,2] chi^2", chi2)
    wopt=ps[0]
    gopt=ps[1]
    dopt=ps[2]
    lopt=ps[3]
    aopt=ps[4]
    bopt=ps[5]
    chis.append(chi2)
    racu.graph(ks, kps, lbs, lps, ps, cs, nplt, pr)
    racu.erG(aopt, bopt)


    """
    fit a [5,2] Pade approximant
    the second resonance guess is 4*Er, 4*Gamma, 
    and the mixing factor epsilon is 1.0
    """

    if pr > 2:
        print ("------------------------------------------------")
    Ers = racu.getErs()
    Gs = racu.getGs()
    Er = Ers[-1] * 20
    G = Gs[-1] * 4
    eg = 0.1
    wg= wopt * 0.05
    gg=0.5*numpy.sqrt(2.0)*(-2*Er + numpy.sqrt(4*Er**2 + G**2))**0.25
    dg=0.5*G/numpy.sqrt(-2*Er + numpy.sqrt(4*Er**2 + G**2))
    ps = [eg, wg, gg, dg, 1]
    cs = ["ppade_52", aopt, bopt]
    lg = lbs[0]/racu.ppade_52(ks[0], ps, cs[1:])
    # preopt epsilon, gamma, delta, omega, and l; keep alpha and beta fixed
    ps = [eg, wg, gg, dg, lg]
    if pr > 2:
        print ("[5,2] pre-guess e, w, g, d, l = "); print ("    ", ps[:5])
    res = minimize(racu.chi, ps, args=(ks, lbs, sigmas, cs), method='BFGS')
    ps = res.x
    eg=ps[0]
    wg=ps[1]
    gg=ps[2]
    dg=ps[3]
    lg=ps[4]
    ps = [eg, wg, gg, dg, lg, aopt, bopt]
    if pr > 2:
        print ("[5,2] pre-opt = full guess e, w, g, d, l, a, b = "); print ("    ", ps)
    cs=["pade_52"]
    res = minimize(racu.chi, ps, args=(ks, lbs, sigmas, cs), method='BFGS',
                   options={'gtol': 1e-6})
    ps = res.x
    if pr > 2:
        print ("[5,2] Full optimization    e, w, g, d, l, a, b = "); print ("    ", ps)
    chi2 = racu.chi(ps, ks, lbs, sigmas, cs)
    if pr > 2:
        print("[5,2] chi^2", chi2)
    eopt = ps[0]
    wopt = ps[1]
    gopt = ps[2]
    dopt = ps[3]
    lopt = ps[4]
    aopt = ps[5]
    bopt = ps[6]
    chis.append(chi2)
    racu.graph(ks, kps, lbs, lps, ps, cs, nplt, pr)
    racu.erG(aopt, bopt)


    """
    fit a [5,3] Pade approximant
    the second resonance guess is 4*Er, 4*Gamma, 
    and the mixing factor epsilon is 1.0
    """

    if pr > 2:
        print ("------------------------------------------------")
    Ers = racu.getErs()
    Gs = racu.getGs()
    Er = Ers[-1] * 20
    G = Gs[-1] * 4
    zg = 0.1
    eg = eopt * 0.05
    wg = wopt * 0.05
    gg = 0.5 * numpy.sqrt(2.0) * (-2 * Er + numpy.sqrt(4 * Er ** 2 + G ** 2)) ** 0.25
    dg = 0.5 * G / numpy.sqrt(-2 * Er + numpy.sqrt(4 * Er ** 2 + G ** 2))
    ps = [zg, eg, wg, 1]
    cs = ["pppade_53", gg, dg, aopt, bopt]
    lg = lbs[0] / racu.pppade_53(ks[0], ps, cs[1:])
    # pre-pre-opt zeta, epsilon, omega, and l; keep alpha, beta, gamma, and delta
    # fixed
    ps = [zg, eg, wg, lg]
    if pr > 2:
        print ("[5,3] pre-pre-guess z, e, w, l = "); print ("    ", ps[:4])
    res = minimize(racu.chi, ps, args=(ks, lbs, sigmas, cs), method='BFGS')
    ps = res.x
    zg = ps[0]
    eg = ps[1]
    wg = ps[2]
    lg = ps[3]
    #pre-opt zeta, epsilon, omega, beta, gamma, and l, alpha and beta fixed
    ps = [zg, eg, wg, gg, dg, lg]
    if pr > 2:
        print ("[5,3] pre-guess z, e, w, g, d, l = "); print ("    ", ps[:6])
    cs = ["ppade_53", aopt, bopt]
    res = minimize(racu.chi, ps, args=(ks, lbs, sigmas, cs), method='BFGS')
    ps = res.x
    zg = ps[0]
    eg = ps[1]
    wg = ps[2]
    gg = ps[3]
    dg = ps[4]
    lg = ps[5]
    ps = [zg, eg, wg, gg, dg, lg, aopt, bopt]
    if pr > 2:
        print ("[5,3] pre-opt = full guess z, e, w, g, d, l, a, b = "); print ("    ",
                                                                              ps)
    cs=["pade_53"]
    res = minimize(racu.chi, ps, args=(ks, lbs, sigmas, cs), method='BFGS',
                   options={'gtol': 1e-6})
    ps = res.x
    if pr > 2:
        print("[5,3] Full optimization   z, e, w, g, d, l, a, b = ");
        print("    ", ps)
    chi2 = racu.chi(ps, ks, lbs, sigmas, cs)
    if pr > 2:
        print("[5,3] chi^2", chi2)
    zopt = ps[0]
    eopt = ps[1]
    wopt = ps[2]
    gopt = ps[3]
    dopt = ps[4]
    lopt = ps[5]
    aopt = ps[6]
    bopt = ps[7]
    chis.append(chi2)
    racu.graph(ks, kps, lbs, lps, ps, cs, nplt, pr)
    racu.erG(aopt, bopt)


    pade = ["Pade [2,1]", "Pade [3,1]", "Pade [3,2]", "Pade [4,2]", "Pade [5,2]",
            "Pade [5,3]"]
    Ers = racu.getErs()
    Gs = racu.getGs()
    print ()
    print ("----------------------------------------------------")
    print ("                Er       Gamma           chi^2      ")
    print ("----------------------------------------------------")
    for i in range(len(pade)):
        print ("{0:s}  {1:8.4f}   {2:11.2e}     {3:11.2e}"\
            .format(pade[i], Ers[i], Gs[i], chis[i]))


In [6]:
print ("R-ACCC")
print (energy_min, energy_max)
rac(lambdas, energys, (energy_min,energy_max), (res_energy,res_width), pr)

R-ACCC
-99.0 0.0
No. of lambda-points:  19
Input energy range: -99.0000 to 0.0000
Largest  energy used is -0.1748 (index 6)
Smallest energy used is -6.4623 (index 18)
No. of points used: 13

----------------------------------------------------
                Er       Gamma           chi^2      
----------------------------------------------------
Pade [2,1]    3.8140      4.08e-01        5.12e-07
Pade [3,1]    3.8136      4.08e-01        5.12e-07
Pade [3,2]    3.8136      4.08e-01        5.12e-07
Pade [4,2]    3.8146      4.09e-01        5.08e-07
Pade [5,2]  -17.3895      3.60e+01        8.98e-07
Pade [5,3]  -17.3895      3.60e+01        2.63e-07
